# Generate histograms of 8bit and 16bit reconstructions

So we can properly answer a reviewer.
They asked "Why is only an 8-bit grayscale used here?"

In [1]:
import platform
import os
import glob
import pandas
import dask
from dask.distributed import Client, LocalCluster
import dask_image.imread
import matplotlib.pyplot as plt
from matplotlib_scalebar.scalebar import ScaleBar
import seaborn
from tqdm.auto import tqdm, trange
import imageio

In [2]:
# Import our own parsing functions which we've added as submodule
from BrukerSkyScanLogfileRuminator.parsing_functions import *

In [80]:
# Set dask temporary folder
# Do this before creating a client: https://stackoverflow.com/a/62804525/323100
import tempfile
if 'Linux' in platform.system():
    tmp = os.path.join(os.sep, 'media', 'habi', 'Fast_SSD')
elif 'Darwin' in platform.system():
    tmp = tempfile.gettempdir()
else:
    if 'anaklin' in platform.node():
        tmp = os.path.join('F:\\')
    else:
        tmp = os.path.join('D:\\')
dask.config.set({'temporary_directory': os.path.join(tmp, 'tmp')})
print('Dask temporary files go to %s' % dask.config.get('temporary_directory'))

Dask temporary files go to /media/habi/Fast_SSD/tmp


In [81]:
# Start cluster and client now, after setting tempdir
cluster = LocalCluster()
client = Client(cluster)

/home/habi/miniconda3/envs/yogita/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 46337 instead
  warnings.warn(


In [82]:
print('You can seee what DASK is doing at "http://localhost:%s/status"' % client.scheduler_info()['services']['dashboard'])

You can seee what DASK is doing at "http://localhost:46337/status"


In [83]:
# # Ignore warnings in the notebook
# import warnings
# warnings.filterwarnings("ignore")

In [84]:
# Set up figure defaults
plt.rc('image', cmap='gray', interpolation='nearest')  # Display all images in b&w and with 'nearest' interpolation
plt.rcParams['figure.figsize'] = (16, 9)  # Size up figures a bit
plt.rcParams['figure.dpi'] = 300

In [85]:
# Setup scale bar defaults
plt.rcParams['scalebar.location'] = 'lower right'
plt.rcParams['scalebar.frameon'] = False
plt.rcParams['scalebar.color'] = 'white'

In [86]:
# Different locations if running either on Linux or Windows
FastSSD = True
# to speed things up significantly
if 'Linux' in platform.system():
    if FastSSD:
        BasePath = os.path.join(os.sep, 'media', 'habi', 'Fast_SSD')
    else:
        BasePath = os.path.join(os.sep, 'home', 'habi', '1272')
elif 'Darwin' in platform.system():
    # First mount smb://resstore.unibe.ch/ana_rs_djonov/data in the Finder
    FastSSD = False
    BasePath = os.path.join('/Volumes/data/')
elif 'Windows' in platform.system():
    if FastSSD:
        BasePath = os.path.join('F:\\')
    else:
        if 'anaklin' in platform.node():
            BasePath = os.path.join('V:\\')
        else:
            BasePath = os.path.join('V:\\')
Root = os.path.join(BasePath, 'Chondrules Space Yogita')
print('We are loading all the data from %s' % Root)

We are loading all the data from /media/habi/Fast_SSD/Chondrules Space Yogita


In [50]:
def get_git_hash():
    '''
    Get the current git hash from the repository.
    Based on http://stackoverflow.com/a/949391/323100 and
    http://stackoverflow.com/a/18283905/323100
    '''
    from subprocess import Popen, PIPE
    import os
    gitprocess = Popen(['git',
                        '--git-dir',
                        os.path.join(os.getcwd(), '.git'),
                        'rev-parse',
                        '--short',
                        '--verify',
                        'HEAD'],
                       stdout=PIPE)
    (output, _) = gitprocess.communicate()
    return output.strip().decode("utf-8")

In [51]:
# Make directory for output
OutPutDir = os.path.join(os.getcwd(), 'Output', get_git_hash())
print('We are saving all the output to %s' % OutPutDir)
os.makedirs(OutPutDir, exist_ok=True)

We are saving all the output to /home/habi/P/Documents/ChondrulesSpaceYogita/Output/ccf0f5d


In [52]:
# Make us a dataframe for saving all that we need
Data = pandas.DataFrame()

In [53]:
# Get *all* log files, unsorted but faster than with glob
print('Searching for all log files in %s' % Root)
Data['LogFile'] = [os.path.join(root, name)
                   for root, dirs, files in os.walk(Root)
                   for name in files
                   if name.endswith((".log"))]

Searching for all log files in /media/habi/Fast_SSD/Chondrules Space Yogita


In [54]:
# Get all folders
Data['Folder'] = [os.path.dirname(f) for f in Data['LogFile']]

In [55]:
#Check for samples which are not yet reconstructed
for c, row in Data.iterrows():
    # Iterate over every 'proj' folder
    if 'proj' in row.Folder:
        if not 'TScopy' in row.Folder and not 'PR' in row.Folder:
            # If there's nothing with 'rec*' on the same level, then tell us        
            if not glob.glob(row.Folder.replace('proj', '*rec*')):
                print('- %s is missing matching reconstructions' % row.LogFile[len(Root)+1:])

In [56]:
Data['XYAlignment'] = [glob.glob(os.path.join(f, '*.csv')) for f in Data['Folder']]

In [57]:
# Check for samples which are missing the .csv-files for the XY-alignment
for c, row in Data.iterrows():
    # Iterate over every 'proj' folder
    if 'proj' in row.Folder:
        if not len(row.XYAlignment):
            if not any(x in row.LogFile for x in ['rectmp.log']):
                # 'rectmp.log' because we only exclude it afterwards :)
                print('- %s has *not* been X/Y aligned' % row.LogFile[len(Root)+1:])

- All-CH2/QuickOverview/proj/All-CH2-Overview.log has *not* been X/Y aligned


In [58]:
# Get rid of all logfiles that we don't want
# For *this* notebook we only want to look at two sets of reconstructions from Qing3
for c, row in Data.iterrows():
    if 'rec' not in row.Folder:
        Data.drop([c], inplace=True)
# Reset dataframe to something that we would get if we only would have loaded the 'rec' files
Data = Data.reset_index(drop=True)

In [59]:
# Generate us some meaningful colums
Data['Sample'] = [l[len(Root)+1:].split(os.sep)[0] for l in Data['LogFile']]
Data['SampleName'] = [sn.split('_')[0] for sn in Data['Sample']]
Data['RecFolder'] = ['_'.join(l[len(Root)+1:].split(os.sep)[1:-1]) for l in Data['LogFile']]

In [60]:
Data[['Sample', 'SampleName', 'RecFolder']]

,Sample,SampleName,RecFolder
0,All-CH2,All-CH2,1umAl0.5_rec_PNG_8bit
1,All-CH2,All-CH2,1umAl0.5_rec_TIFF_16bit
2,All-CH2,All-CH2,1umAl0.5Cu0.038_rec_PNG_8bit
3,All-CH2,All-CH2,1umAl0.5Cu0.038_rec_TIFF_16bit
4,All-CH2,All-CH2,QuickOverview_rec
5,All-CH2,All-CH2,QuickOverview_rec_PNG_8bit
6,All-CH2,All-CH2,QuickOverview_rec_TIFF_16bit
7,CK3A,CK3A,rec_bottom_PNG_8bit
8,CK3A,CK3A,rec_bottom_TIFF_16bit
9,CK3B,CK3B,rec_top_PNG_8bit


In [61]:
# Get the file names of the reconstructions
Data['Reconstructions'] = [sorted(glob.glob(os.path.join(f, '*rec0*.*'))) for f in Data['Folder']]
Data['Number of reconstructions'] = [len(r) for r in Data.Reconstructions]

In [62]:
# Drop samples which have either not been reconstructed yet or of which we deleted the reconstructions with
# `find . -name "*rec*.png" -type f -mtime +333 -delete`
# Based on https://stackoverflow.com/a/13851602
# for c,row in Data.iterrows():
#     if not row['Number of reconstructions']:
#         print('%s contains no PNG files, we might be currently reconstructing it' % row.Folder)
Data = Data[Data['Number of reconstructions'] > 0]
Data.reset_index(drop=True, inplace=True)
print('We have %s folders with reconstructions' % (len(Data)))

We have 7 folders with reconstructions


In [63]:
Data

,LogFile,Folder,XYAlignment,Sample,SampleName,RecFolder,Reconstructions,Number of reconstructions
0,/media/habi/Fast_SSD/Chondrules Space Yogita/A...,/media/habi/Fast_SSD/Chondrules Space Yogita/A...,[],All-CH2,All-CH2,1umAl0.5_rec_PNG_8bit,[/media/habi/Fast_SSD/Chondrules Space Yogita/...,1678
1,/media/habi/Fast_SSD/Chondrules Space Yogita/A...,/media/habi/Fast_SSD/Chondrules Space Yogita/A...,[],All-CH2,All-CH2,1umAl0.5Cu0.038_rec_PNG_8bit,[/media/habi/Fast_SSD/Chondrules Space Yogita/...,1621
2,/media/habi/Fast_SSD/Chondrules Space Yogita/A...,/media/habi/Fast_SSD/Chondrules Space Yogita/A...,[],All-CH2,All-CH2,QuickOverview_rec_PNG_8bit,[/media/habi/Fast_SSD/Chondrules Space Yogita/...,571
3,/media/habi/Fast_SSD/Chondrules Space Yogita/C...,/media/habi/Fast_SSD/Chondrules Space Yogita/C...,[],CK3A,CK3A,rec_bottom_PNG_8bit,[/media/habi/Fast_SSD/Chondrules Space Yogita/...,542
4,/media/habi/Fast_SSD/Chondrules Space Yogita/C...,/media/habi/Fast_SSD/Chondrules Space Yogita/C...,[],CK3B,CK3B,rec_top_PNG_8bit,[/media/habi/Fast_SSD/Chondrules Space Yogita/...,624
5,/media/habi/Fast_SSD/Chondrules Space Yogita/Q...,/media/habi/Fast_SSD/Chondrules Space Yogita/Q...,[],Qing3,Qing3,rec_bottom_JPG_08bit,[/media/habi/Fast_SSD/Chondrules Space Yogita/...,423
6,/media/habi/Fast_SSD/Chondrules Space Yogita/Q...,/media/habi/Fast_SSD/Chondrules Space Yogita/Q...,[],Qing3,Qing3,rec_bottom_PNG_08bit,[/media/habi/Fast_SSD/Chondrules Space Yogita/...,701


In [64]:
# Get scanning parameters to doublecheck from logfiles
Data['Scanner'] = [scanner(log) for log in Data['LogFile']]
Data['Voltage'] = [voltage(log) for log in Data['LogFile']]
Data['Current'] = [current(log) for log in Data['LogFile']]
Data['Voxelsize'] = [pixelsize(log, rounded=True) for log in Data['LogFile']]
Data['ProjectionSize'] = [projection_size(log) for log in Data['LogFile']]
Data['Exposuretime'] = [exposure(log) for log in Data['LogFile']]
Data['Averaging'] = [averaging(log) for log in Data['LogFile']]
Data['Stacks'] = [stacks(log) for log in Data['LogFile']]
Data['RotationStep'] = [rotationstep(log) for log in Data['LogFile']]
Data['Scan date'] = [scandate(log) for log in Data['LogFile']]
Data['Scan time'] = [duration(log) for log in Data['LogFile']]

In [65]:
Data.Voltage.unique()

array([70., 90.])

In [66]:
Data.Current.unique()

array([142., 111.])

In [67]:
Data.Averaging.unique()

array([3, False], dtype=object)

In [68]:
Data.Voxelsize.unique()

array([1.  , 3.  , 1.65])

In [69]:
# Get reconstruction parameters to doublecheck from logfiles
Data['Grayvalue'] = [reconstruction_grayvalue(log) for log in Data['LogFile']]
Data['RingartefactCorrection'] = [ringremoval(log) for log in Data['LogFile']]
Data['BeamHardeningCorrection'] = [beamhardening(log) for log in Data['LogFile']]
Data['DefectPixelMasking'] = [defectpixelmasking(log) for log in Data['LogFile']]

In [70]:
Data['Grayvalue'].unique()

array([1.4     , 0.8     , 1.3     , 2.077912, 5.      , 1.8     ])

In [71]:
Data['RingartefactCorrection'].unique()

array([20, 14])

In [72]:
for c,row in Data.iterrows():
    if not row.RingartefactCorrection:
        print(row.Sample, row.RingartefactCorrection)

In [73]:
Data[['Sample', 'RecFolder', 'Voxelsize', 'Grayvalue', 'RingartefactCorrection', 'BeamHardeningCorrection', 'DefectPixelMasking']]

,Sample,RecFolder,Voxelsize,Grayvalue,RingartefactCorrection,BeamHardeningCorrection,DefectPixelMasking
0,All-CH2,1umAl0.5_rec_PNG_8bit,1.00,1.400000,20,0,50
1,All-CH2,1umAl0.5Cu0.038_rec_PNG_8bit,1.00,0.800000,14,0,50
2,All-CH2,QuickOverview_rec_PNG_8bit,3.00,1.300000,14,0,50
3,CK3A,rec_bottom_PNG_8bit,1.65,2.077912,14,0,50
4,CK3B,rec_top_PNG_8bit,1.65,5.000000,14,0,50
5,Qing3,rec_bottom_JPG_08bit,1.65,1.800000,14,0,50
6,Qing3,rec_bottom_PNG_08bit,1.65,1.800000,14,0,50


In [74]:
Data.tail()

,LogFile,Folder,XYAlignment,Sample,SampleName,RecFolder,Reconstructions,Number of reconstructions,Scanner,Voltage,...,Exposuretime,Averaging,Stacks,RotationStep,Scan date,Scan time,Grayvalue,RingartefactCorrection,BeamHardeningCorrection,DefectPixelMasking
2,/media/habi/Fast_SSD/Chondrules Space Yogita/A...,/media/habi/Fast_SSD/Chondrules Space Yogita/A...,[],All-CH2,All-CH2,QuickOverview_rec_PNG_8bit,[/media/habi/Fast_SSD/Chondrules Space Yogita/...,571,SkyScan 1272,70.0,...,1516,False,1,0.6,2020-02-18 15:53:26,1305.0,1.300000,14,0,50
3,/media/habi/Fast_SSD/Chondrules Space Yogita/C...,/media/habi/Fast_SSD/Chondrules Space Yogita/C...,[],CK3A,CK3A,rec_bottom_PNG_8bit,[/media/habi/Fast_SSD/Chondrules Space Yogita/...,542,SkyScan 1272,70.0,...,3900,3,1,0.1,2021-04-09 01:06:41,33216.0,2.077912,14,0,50
4,/media/habi/Fast_SSD/Chondrules Space Yogita/C...,/media/habi/Fast_SSD/Chondrules Space Yogita/C...,[],CK3B,CK3B,rec_top_PNG_8bit,[/media/habi/Fast_SSD/Chondrules Space Yogita/...,624,SkyScan 1272,70.0,...,3900,3,1,0.1,2021-04-09 10:36:03,33211.0,5.000000,14,0,50
5,/media/habi/Fast_SSD/Chondrules Space Yogita/Q...,/media/habi/Fast_SSD/Chondrules Space Yogita/Q...,[],Qing3,Qing3,rec_bottom_JPG_08bit,[/media/habi/Fast_SSD/Chondrules Space Yogita/...,423,SkyScan 1272,70.0,...,5300,3,1,0.1,2020-12-12 09:51:42,43734.0,1.800000,14,0,50
6,/media/habi/Fast_SSD/Chondrules Space Yogita/Q...,/media/habi/Fast_SSD/Chondrules Space Yogita/Q...,[],Qing3,Qing3,rec_bottom_PNG_08bit,[/media/habi/Fast_SSD/Chondrules Space Yogita/...,701,SkyScan 1272,70.0,...,5300,3,1,0.1,2020-12-12 09:51:42,43734.0,1.800000,14,0,50


In [75]:
# Load all reconstructions into the dataframe
Reconstructions = [dask_image.imread.imread(os.path.join(f,'*rec0*.*')) for f in Data['Folder']]

In [76]:
# How big are the datasets?
Data['Size'] = [rec.shape for rec in Reconstructions]

In [77]:
Data[['Sample', 'RecFolder', 'Size']]

,Sample,RecFolder,Size
0,All-CH2,1umAl0.5_rec_PNG_8bit,"(1678, 1836, 1836, 3)"
1,All-CH2,1umAl0.5Cu0.038_rec_PNG_8bit,"(1621, 1804, 1804, 3)"
2,All-CH2,QuickOverview_rec_PNG_8bit,"(571, 660, 660, 3)"
3,CK3A,rec_bottom_PNG_8bit,"(542, 744, 744, 3)"
4,CK3B,rec_top_PNG_8bit,"(624, 812, 812, 3)"
5,Qing3,rec_bottom_JPG_08bit,"(540, 800, 788, 3)"
6,Qing3,rec_bottom_PNG_08bit,"(701, 800, 788, 3)"


In [78]:
# The three cardinal directions
directions = ['Axial',
              'Coronal',
              'Sagittal']

In [79]:
# Read or calculate the middle slices, put them into the dataframe and save them to disk
for d, direction in enumerate(directions):
    Data['Mid_' + direction] = [None] * len(Reconstructions)
for c, row in tqdm(Data.iterrows(), desc='Middle images', total=len(Data), leave=False):
    for d, direction in tqdm(enumerate(directions),
                             desc='%s/%s' % (row['Sample'], row['RecFolder']),
                             leave=False,
                             total=len(directions)):
        outfilepath = os.path.join(os.path.dirname(row['Folder']),
                                   '%s.%s.Middle.%s.png' % (row['Sample'],
                                                            row['RecFolder'],
                                                            direction))
        if os.path.exists(outfilepath):
            Data.at[c, 'Mid_' + direction] = dask_image.imread.imread(outfilepath).squeeze()
        else:
            # Generate requested axial view
            if 'Axial' in direction:
                Data.at[c, 'Mid_' + direction] = Reconstructions[c][Data['Size'][c][0] // 2].compute().squeeze()
            if 'Sagittal' in direction:
                Data.at[c, 'Mid_' + direction] = Reconstructions[c][:, Data['Size'][c][1] // 2, :].compute().squeeze()
            if 'Coronal' in direction:
                Data.at[c, 'Mid_' + direction] = Reconstructions[c][:, :, Data['Size'][c][2] // 2].compute().squeeze()
            # Save the calculated 'direction' view to disk
            imageio.imwrite(outfilepath, (Data.at[c, 'Mid_' + direction]))

Middle images:   0%|          | 0/7 [00:00<?, ?it/s]

All-CH2/1umAl0.5_rec_PNG_8bit:   0%|          | 0/3 [00:00<?, ?it/s]

All-CH2/1umAl0.5Cu0.038_rec_PNG_8bit:   0%|          | 0/3 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
# Show middle slices
for c, row in tqdm(Data.iterrows(),
                   desc='Saving middle images overview',
                   total=len(Data),
                   leave=False):
    outfilepath = os.path.join(os.path.dirname(row['Folder']),
                               '%s.%s.MiddleSlices.png' % (row['Sample'], row['Scan']))
    if not os.path.exists(outfilepath):    
        for d, direction in tqdm(enumerate(directions),
                                 desc='%s/%s' % (row['Sample'], row['Scan']),
                                 leave=False,
                                 total=len(directions)):
            plt.subplot(1, 3, d + 1)
            plt.imshow(row['Mid_' + direction].squeeze())
            if d == 0:
                plt.axhline(row.Size[1] // 2, c=seaborn.color_palette()[0])
                plt.axvline(row.Size[2] // 2, c=seaborn.color_palette()[1])
                plt.gca().add_artist(ScaleBar(row['Voxelsize'],
                                              'um',
                                              color=seaborn.color_palette()[2]))
            elif d == 1:
                plt.axhline(row.Size[0] // 2, c=seaborn.color_palette()[2])
                plt.axvline(row.Size[d] // 2, c=seaborn.color_palette()[1])
                plt.gca().add_artist(ScaleBar(row['Voxelsize'],
                                              'um',
                                              color=seaborn.color_palette()[0]))
            else:
                plt.axhline(row.Size[0] // 2, c=seaborn.color_palette()[2])
                plt.axvline(row.Size[d] // 2, c=seaborn.color_palette()[0])
                plt.gca().add_artist(ScaleBar(row['Voxelsize'],
                                              'um',
                                              color=seaborn.color_palette()[1]))
            plt.title('%s - %s' % (os.path.join(row['Sample'], row['RecFolder']),
                                   direction + ' Middle slice'))
            plt.axis('off')
            plt.savefig(outfilepath,
                        transparent=True,
                        bbox_inches='tight')
        plt.show()

In [ ]:
# Read or calculate the directional MIPs, put them into the dataframe and save them to disk
for d, direction in enumerate(directions):
    Data['MIP_' + direction] = [None] * len(Reconstructions)
for c, row in tqdm(Data.iterrows(), desc='MIPs', total=len(Data), leave=False):
    for d, direction in tqdm(enumerate(directions),
                             desc='%s/%s' % (row['Sample'], row['RecFolder']),
                             leave=False,
                             total=len(directions)):
        outfilepath = os.path.join(os.path.dirname(row['Folder']),
                                   '%s.%s.MIP.%s.png' % (row['Sample'],
                                                         row['RecFolder'],
                                                         direction))
        if os.path.exists(outfilepath):
            Data.at[c, 'MIP_' + direction] = dask_image.imread.imread(outfilepath).squeeze()
        else:
            # Generate MIP
            Data.at[c, 'MIP_' + direction] = Reconstructions[c].max(axis=d).compute().squeeze()
            # Save it out
            imageio.imwrite(outfilepath, Data.at[c, 'MIP_' + direction].astype('uint8'))

In [ ]:
# Show MIP slices
for c, row in tqdm(Data.iterrows(),
                   desc='Saving MIP images overview',
                   total=len(Data),
                   leave=False):
    outfilepath = os.path.join(os.path.dirname(row['Folder']),
                               '%s.%s.MIPs.png' % (row['Sample'], row['RecFolder']))
    if not os.path.exists(outfilepath):    
        for d, direction in tqdm(enumerate(directions),
                                          desc='%s/%s' % (row['Sample'], row['Scan']),
                                          leave=False,
                                          total=len(directions)):
            plt.subplot(1, 3, d + 1)
            plt.imshow(row['MIP_' + direction].squeeze())
            plt.gca().add_artist(ScaleBar(row['Voxelsize'],
                                          'um'))
            plt.title('%s - %s' % (os.path.join(row['Sample'], row['RecFolder']),
                                   direction + ' MIP'))
            plt.axis('off')
        plt.savefig(outfilepath,
                    transparent=True,
                    bbox_inches='tight')
        plt.show()

In [ ]:
# Calculate and plot each histogram of the reconstructions
for c, row in Data.iterrows():
    plt.hist(Reconstructions[c].ravel().compute(),
             log=False,
             bins='sturges',
             histtype='bar',
             label=Reconstructions[c].dtype)
    plt.legend()
    plt.title(os.path.join(row.Sample, row.RecFolder))
    plt.savefig(os.path.join(Root, row.Sample, '.'.join((row.Sample, row.RecFolder, 'Histogram', 'png'))),
                bbox_inches='tight')
    plt.savefig(os.path.join(OutPutDir, '.'.join((row.Sample, row.RecFolder, 'Histogram', 'png'))),
                bbox_inches='tight')    
    plt.show()

In [ ]:
print('Saved all histograms to %s' % OutPutDir)